In [1]:
# requirements 
%pip install pandas --quiet 
%pip install flask requests --quiet
%pip install ollama --quiet
%pip install streamlit streamlit-chat --quiet
%pip install langchain --quiet
%pip install faiss-cpu --quiet
%pip install transformers --quiet
%pip install ctransformers --quiet
%pip install requests --quiet
%pip install -U langchain-community --quiet



print("Requirements installed successfully!")


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider 

In [2]:
# imports
print("Starting imports...")
import pandas as pd
from flask import Flask, request, jsonify
import ollama
import json
import requests
import re
import streamlit as st 
from streamlit_chat import message
import tempfile
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

print("Successfully imported!")

Starting imports...


/Users/rachelsu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2024-11-23 13:00:28.581 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Successfully imported!


In [3]:
def load_llm():
    llm = CTransformers(
        model = "res\llama-2-7b-chat.ggmlv3.q4_1.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.9
    )
    return llm


In [ ]:
# # Load the dataset into a df
# carsDF = pd.read_csv('vehicles.csv')

In [ ]:
# # view dataset 
# carsDF.head()

In [ ]:
# # Normalize text values to title case for consistency
# carsDF['Make'] = carsDF['Make'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase
# carsDF['Model'] = carsDF['Model'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase


In [ ]:
# # Verify normalization
# print(carsDF[['Make', 'Model']])

In [ ]:
# # Data Type Correction
# # Convert numerical columns
# carsDF['Year'] = carsDF['Year'].astype(int)
# carsDF['SellingPrice'] = carsDF['SellingPrice'].astype(float)
# carsDF['Doors'] = carsDF['Doors'].astype(int)


In [ ]:
# carsDF['PriceRange'] = pd.cut(carsDF['SellingPrice'], 
#                               bins=[0, 10000, 20000, 30000, 50000, 100000],
#                               labels=['Under 10K', '10K-20K', '20K-30K', '30K-50K', 'Above 50K'])


In [5]:
# Extract preferences from user's message
def extract_preferences(message, carsDF):
    preferences = {}

    if "new" in message.lower() or "used" in message.lower():
        preferences['car_type'] = 'New' if 'new' in message.lower() else 'Used'

    if 'year' in message.lower():
        year = [int(word) for word in message.split() if word.isdigit() and len(word) == 4]
        if year:
            preferences['year'] = year[0]

    if any(make in message.lower() for make in carsDF['Make'].str.lower().unique()):
        preferences['make'] = next(make.title() for make in carsDF['Make'].str.lower().unique() if make in message.lower())

    # Extract price range (e.g., "under $20,000", "between $20,000 and $30,000")
    price_under = re.search(r'under\s*\$?(\d{1,3}(?:,\d{3})*)', message, re.IGNORECASE)
    price_between = re.search(r'between\s*\$?(\d{1,3}(?:,\d{3})*)\s*and\s*\$?(\d{1,3}(?:,\d{3})*)', message, re.IGNORECASE)

    if price_under:
        max_price = int(price_under.group(1).replace(',', ''))
        preferences['max_price'] = max_price
    elif price_between:
        min_price = int(price_between.group(1).replace(',', ''))
        max_price = int(price_between.group(2).replace(',', ''))
        preferences['min_price'] = min_price
        preferences['max_price'] = max_price

    # Extract mileage
    if "low mileage" in message.lower():
        preferences['max_miles'] = 50000  # Assume "low mileage" means under 50,000 miles
    mileage = re.search(r'(\d{1,3}(?:,\d{3})*)\s*miles', message, re.IGNORECASE)
    if mileage:
        preferences['max_miles'] = int(mileage.group(1).replace(',', ''))

    # Extract exterior color preference
    colors = carsDF['Ext_Color_Generic'].dropna().apply(str).str.lower().unique()
    for color in colors:
        if color in message.lower():
            preferences['exterior_color'] = color.title()
            break

    # Extract drivetrain preference
    drivetrains = ['AWD', 'FWD', 'RWD', '4WD']
    for drivetrain in drivetrains:
        if drivetrain.lower() in message.lower():
            preferences['drivetrain'] = drivetrain.upper()
            break

    return preferences

In [6]:
# Function to get matching cars based on preferences
def get_matching_cars(preferences, carsDF):
    filtered_cars = carsDF

    # Apply each preference as a filter
    if 'car_type' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Type'].str.lower() == preferences['car_type'].lower()]

    if 'year' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Year'] == preferences['year']]

    if 'make' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Make'].str.upper() == preferences['make'].upper()]

    if 'min_price' in preferences and 'max_price' in preferences:
        filtered_cars = filtered_cars[
            (filtered_cars['SellingPrice'] >= preferences['min_price']) & 
            (filtered_cars['SellingPrice'] <= preferences['max_price'])
        ]
    elif 'max_price' in preferences:
        filtered_cars = filtered_cars[filtered_cars['SellingPrice'] <= preferences['max_price']]

    if 'max_miles' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Miles'] <= preferences['max_miles']]

    if 'exterior_color' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Ext_Color_Generic'].str.lower() == preferences['exterior_color'].lower()]

    if 'drivetrain' in preferences:
        filtered_cars = filtered_cars[filtered_cars['Drivetrain'].str.upper() == preferences['drivetrain'].upper()]

    # Return the top 3 matching cars
    if not filtered_cars.empty:
        return filtered_cars[['Make', 'Model', 'Year', 'VIN', 'SellingPrice']].head(3)
    else:
        return None


In [7]:
# Streamlit UI
st.title("Matador Car Match Chatbot")

2024-11-23 13:01:01.391 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 13:01:01.454 
  command:

    streamlit run /Users/rachelsu/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-11-23 13:01:01.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [8]:
# Sidebar for CSV file upload
csv_data = st.sidebar.file_uploader("vehicles.csv", type="csv")

2024-11-23 13:01:04.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 13:01:04.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 13:01:04.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 13:01:04.535 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 13:01:04.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
if csv_data is not None:
    # Handle uploaded CSV file with temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp_file:
        tmp_file.write(csv_data.getvalue())
        tmp_file_path = tmp_file.name

    # Load CSV data using pandas directly
    carsDF = pd.read_csv(tmp_file_path)

    # Load LLM
    llm = load_llm()

    # Chat UI
    if 'user' not in st.session_state:
        st.session_state['user'] = ["Hey there"]

    if 'assistant' not in st.session_state:
        st.session_state['assistant'] = ["Hello, I am Matador Chatbot! How can I help you find your car today?"]

    container = st.container()

    with container:
        with st.form(key='car_form', clear_on_submit=True):
            user_input = st.text_input("", placeholder="Enter your car preferences...", key='input')
            submit = st.form_submit_button(label='Submit')

        if submit:
            # Append user message
            st.session_state['user'].append(user_input)

            # Extract preferences and find matching cars
            preferences = extract_preferences(user_input, carsDF)
            matching_cars = get_matching_cars(preferences, carsDF)

            # LLM response
            if matching_cars is not None:
                llm_response = f"Here are the top 3 cars that match your preferences:\n\n{matching_cars.to_string(index=False)}"
            else:
                llm_response = "I couldn't find any cars matching your preferences. Could you refine your criteria?"

            st.session_state['assistant'].append(llm_response)

    # Display chat messages
    if st.session_state['assistant']:
        for i in range(len(st.session_state['assistant'])):
            message(st.session_state["user"][i], is_user=True, key=str(i) + '_user')
            message(st.session_state["assistant"][i], key=str(i))        

In [ ]:
# # Sample message to test the get_matching_cars function
# #test_message = "I'm looking for a used car from 2019, preferably under $30,000 with low mileage."
# test_message = "I'm looking for a 2014 Honda with high mileage."

# # Get matching cars
# matching_cars = get_matching_cars(test_message)

# # Display the result
# if matching_cars is not None:
#     print("Matching Cars:")
#     print(matching_cars)
# else:
#     print("No cars match the specified preferences.")

